## Create Genomics Store Supervisor Agent

**Updated Version - With Quality-Based Filtering**

In this notebook we create the Genomics Store Supervisor Agent that will interact with **genomicsvariantstore** and **genomicsannotationstore** directly using AWS HealthOmics APIs and Athena queries. This version includes comprehensive quality filtering and uses real HealthOmics store queries with dynamic SQL generation capabilities.

### Key Features:
- Direct queries to genomicsvariantstore and genomicsannotationstore
- Dynamic SQL query generation using AI
- Proper VEP and ClinVar annotation handling
- **Quality-based filtering**: All variant analyses automatically apply:
  - `v.qual > 30 AND contains(v.filters, 'PASS')` for standard quality
  - `v.qual > 50 AND contains(v.filters, 'PASS')` for high quality
- No DynamoDB pipeline tracking dependency
- Real genomic variant analysis functions
- No LIMIT clauses - complete result sets

#### Install required packages for genomics store analysis

In [1]:
!pip install --upgrade boto3
!pip install pandas numpy

#### Import required libraries and initialize AWS clients

In [2]:
import boto3
import json
import time
from datetime import datetime
import pandas as pd
import numpy as np

# Import genomics store modules
from genomics_store_interpreters import (
    genomics_store_agent_name,
    genomics_store_agent_description,
    genomics_store_agent_instruction,
    genomics_store_agent_tools,
    model
)

from genomics_store_functions import (
    REGION,
    ACCOUNT_ID,
    VARIANT_STORE_NAME,
    ANNOTATION_STORE_NAME,
    LAKE_FORMATION_DATABASE,
    get_stores_information,
    get_available_samples_from_variant_store,
    execute_dynamic_query
)

# Initialize AWS clients
session = boto3.Session()
region = session.region_name or '<YOUR_REGION>'
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']

print(f"AWS Account: {account_id}")
print(f"Region: {region}")
print(f"Variant Store: {VARIANT_STORE_NAME}")
print(f"Annotation Store: {ANNOTATION_STORE_NAME}")
print(f"Database: {LAKE_FORMATION_DATABASE}")
print("\n✅ No DynamoDB dependency - using direct store queries")
print("✅ Quality-based filtering enabled")

✅ AWS configuration detected - Region: <YOUR_REGION>, Account: <YOUR_ACCOUNT_ID>
✅ Athena client initialized
✅ Bedrock client initialized
✅ HealthOmics client initialized
✅ Glue client initialized
Region: <YOUR_REGION>
Account ID: <YOUR_ACCOUNT_ID>
Variant Store: genomicsvariantstore
Annotation Store: genomicsannotationstore
Database: default
✅ Genomics store agent tools defined
✅ Configured for:
   - Variant Store: genomicsvariantstore
   - Annotation Store: genomicsannotationstore
   - Database: default
   - Region: <YOUR_REGION>
   - Tools: 5
AWS Account: <YOUR_ACCOUNT_ID>
Region: <YOUR_REGION>
Variant Store: genomicsvariantstore
Annotation Store: genomicsannotationstore
Database: default

✅ No DynamoDB dependency - using direct store queries
✅ Quality-based filtering enabled


#### Initialize Bedrock AgentCore client

In [3]:
# Initialize Bedrock AgentCore client
bedrock_agent_client = boto3.client('bedrock-agent', region_name=region)
bedrock_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region)

print("✅ Bedrock AgentCore clients initialized")

✅ Bedrock AgentCore clients initialized


In [4]:
from magic_helper import register_cell_magic

#### Create Strands Agent for Genomics Store Analysis

In [5]:
%%write_and_run vcf_agent_supervisor.py

import boto3
import json
import uuid
import requests
from typing import Dict, Any
from strands import Agent, tool
from strands.models import BedrockModel

from genomics_store_interpreters import *

# 1: vcf_interpreter_agent tool

@tool
def vcf_supervisor_agent(query: str) -> str:
    """
    Create vcf interpreter agent with lake formation tables using Strands framework

    Args:
        query: An information request from the vcf database

    Returns:
        A summary of the understanding of the user's query and the response.
    """
    try:
        vcf_sup_agent = Agent(
            model=model,
            tools=genomics_store_agent_tools,
            system_prompt=genomics_store_agent_instruction
        )
        vcf_agent_response = vcf_sup_agent(query)
        print("VCF Supervisor agent Response:")
        print(vcf_agent_response)
        return vcf_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

# Define orchestrator agent configuration below

agent_name = "vcf-agent-supervisor"
agent_description = "VCF supervisor agent for cinical insights disocvery"
agent_instruction = """You are an advanced genomics analysis assistant specialized in clinical variant interpretation using AWS HealthOmics genomicsvariantstore and genomicsannotationstore.

Your primary focus is on clinically actionable genomic analysis with these specialized capabilities:

CORE CLINICAL TOOLS:
1. **Gene-Specific Analysis** (query_variants_by_gene): For targeted gene panels, cancer genes (BRCA1/2, TP53), pharmacogenes (CYP2D6, CYP2C19)
2. **Chromosomal Analysis** (query_variants_by_chromosome): For chromosomal abnormalities, CNV analysis, specific genomic regions
3. **Population Frequency Analysis** (analyze_allele_frequencies): For rare disease variants, population genetics, allele frequency comparisons with 1000 Genomes
4. **Sample Comparison** (compare_sample_variants): For family studies, cohort analysis, population stratification
5. **Dynamic Queries** (execute_dynamic_genomics_query): For complex clinical questions requiring custom SQL

CLINICAL DECISION SUPPORT:
- Always apply quality filtering (qual > 30 AND PASS filters)
- Prioritize variants by clinical significance and functional impact
- Include 1000 Genomes frequency data for population context
- Provide actionable clinical interpretations

TOOL SELECTION STRATEGY:
1. **For specific genes**: Use query_variants_by_gene (e.g., "BRCA1 variants", "CYP2D6 pharmacogenomics")
2. **For chromosomal regions**: Use query_variants_by_chromosome (e.g., "chromosome 17 variants", "chr13:32000000-33000000")
3. **For frequency analysis**: Use analyze_allele_frequencies (e.g., "rare variants", "population frequencies")
4. **For cohort studies**: Use compare_sample_variants (e.g., "compare samples", "family analysis")
5. **For complex queries**: Use execute_dynamic_genomics_query

Remember: Focus on clinically actionable insights that can inform patient care, genetic counseling, and treatment decisions.
"""

# Define the model
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0", 
    region_name=region,
    temperature=0.1,
    streaming=False
)

# Instantiate the orchestrator agent
try:
    orchestrator = Agent(
        model=bedrock_model,
        system_prompt=agent_instruction,
        callback_handler=None,
        # associate sub-agents to supervisor
        tools=[vcf_supervisor_agent]
    )
    print(f"Successfully created orchestrator agent: {agent_name}")
except Exception as e:
    print(f"Error creating agent: {e}")
    raise

Successfully created orchestrator agent: vcf-agent-supervisor


In [6]:
# ---------------------------- Sample Question Bank --------------------------------------------

# VCF Agent Questions
vcf_agent_query_1 = "Find all pathogenic variants in BRCA1 and BRCA2 genes for patient NA21144"
vcf_agent_query_2 = "Analyze pathogenic variants in chromosome 17 in patient NA21144 and let me know any clinical followup need to be carried out"
vcf_agent_query_3 = "What are the rare variants (frequency < 0.01) in patient NA21144?"
vcf_agent_query_4 = "Compare variant profiles between samples NA21144 and NA21135"
vcf_agent_query_5 = "Find pharmacogenomic variants in CYP2D6 gene for all samples"
vcf_agent_query_6 = "Howmany patients are in present cohort?"

# -----------------------------------------------------------------------------------------
test_query = vcf_agent_query_1 # Change value here to test different questions

print(f"Testing orchestrator agent with query: {test_query}")
print("=" * (39 + len(test_query)))

try:
    # Run the agent
    response = orchestrator(test_query)
    
except Exception as e:
    print(f"Error during agent execution: {e}")
    import traceback
    traceback.print_exc()

Testing orchestrator agent with query: Find all pathogenic variants in BRCA1 and BRCA2 genes for patient NA21144
I'll help you find pathogenic and likely pathogenic variants in BRCA1 and BRCA2 genes for patient NA21144. Let me query these genes specifically for this patient.
Tool #1: query_variants_by_gene
Executing query on database '<YOUR_AWS_PROFILE>': 
        WITH variant_annotations AS (
            SELECT 
                v.sampleid,
                v.contigname,
                v.start,
                v.referenceallele,
                v.alternatealleles[1] as alternate_allele,
                v.qual,
                v.depth,
                v.information['af'] as allele_frequency_1000g,
                v.filters[1] as filter_status,
                
                CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene,
                CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
      

In [11]:
response = orchestrator("Can you compare the pathogenicity of patients in my cohort?")

I'll compare the pathogenicity profiles of patients NA21135 and NA21144 to analyze their genetic risk burden. Let me use the sample comparison tool to get a comprehensive analysis of their variants.
Tool #1: compare_sample_variants
Executing query on database '<YOUR_AWS_PROFILE>': 
        WITH sample_variants AS (
            SELECT 
                v.sampleid,
                v.qual,
                v.depth,
                v.referenceallele,
                v.alternatealleles[1] as alternate_allele,
                v.filters[1] as filter_status,
                
                CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene,
                CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impact,
                
                a.attributes['CLNSIG'] as clinical_significance,
                split_part(a.attributes['GENEINFO'], ':', 1) as clinvar_gene
                
            FROM genomicsvar

In [9]:
response = orchestrator("Analyze the patients cohort and provide a comprehensive clinical summary including: individual risk stratification, population-level insights, shared pathogenic variants, personalized medicine recommendations, and clinical prioritization for genetic counseling. And let me know how do you assess the risk and prioritization?")

I'll analyze the cohort for pathogenic and likely pathogenic variants across all samples. This will provide valuable clinical insights about disease-causing mutations in the patient population.

To get a comprehensive view of clinically significant variants, I'll use the allele frequency analysis tool which will identify pathogenic and likely pathogenic variants across all samples in the cohort.
Tool #1: analyze_allele_frequencies
Executing query on database '<YOUR_AWS_PROFILE>': 
        WITH variant_data AS (
            SELECT 
                v.sampleid,
                v.contigname,
                v.start,
                v.referenceallele,
                v.alternatealleles[1] as alternate_allele,
                v.qual,
                v.depth,
                
                CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].symbol END as vep_gene,
                CASE WHEN cardinality(v.annotations.vep) > 0 THEN v.annotations.vep[1].impact END as vep_impa

In [10]:
response = orchestrator("Analyze the key BRCA1 pathogenic variant allelic frequency in 1000 genome project and check my patients had any of those variants, provide me the summary?")

I'll analyze pathogenic BRCA1 variants in the 1000 Genomes Project and check if patients NA21135 and NA21144 have any of these variants. Let me break this down into steps:

1. First, I'll query BRCA1 variants with clinical annotations and frequency data
2. Then I'll specifically check if samples NA21135 and NA21144 have any pathogenic BRCA1 variants
3. Finally, I'll provide allele frequencies and clinical significance details

Let's start with the BRCA1 variant analysis:
Tool #1: query_variants_by_gene
Executing query on database '<YOUR_AWS_PROFILE>': 
        WITH variant_annotations AS (
            SELECT 
                v.sampleid,
                v.contigname,
                v.start,
                v.referenceallele,
                v.alternatealleles[1] as alternate_allele,
                v.qual,
                v.depth,
                v.information['af'] as allele_frequency_1000g,
                v.filters[1] as filter_status,
                
                CASE WHEN card

## Deploy Agent to Bedrock AgentCore

### Preparing your agent for deployment on AgentCore Runtime

In [12]:
%%writefile vcf_agent_marker.py

from strands import Agent, tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

# Import everything from vcf_interpreters
from genomics_store_interpreters import *

boto_session = Session()
# Define the model
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0", 
    region_name=region,
    temperature=0.1,
    streaming=False
)

# Define the supervisor agent tool
@tool
def vcf_supervisor_agent(query: str) -> str:
    """
    Create vcf interpreter agent with lake formation tables using Strands framework

    Args:
        query: An information request from the vcf database

    Returns:
        A summary of the understanding of the user's query and the response.
    """
    try:
        vcf_sup_agent = Agent(
            model=model,
            tools=genomics_store_agent_tools,
            system_prompt=genomics_store_agent_instruction
        )
        vcf_agent_response = vcf_sup_agent(query)
        print("VCF Supervisor agent Response:")
        print(vcf_agent_response)
        return vcf_agent_response
    except Exception as e:
        print(f"Error creating agent: {e}")
        raise

# Define orchestrator agent configuration below

agent_name = "vcf-agent-supervisor"
agent_description = "VCF supervisor agent for cinical insights disocvery"
agent_instruction = """You are an advanced genomics analysis assistant specialized in clinical variant interpretation using AWS HealthOmics genomicsvariantstore and genomicsannotationstore.

Your primary focus is on clinically actionable genomic analysis with these specialized capabilities:

CORE CLINICAL TOOLS:
1. **Gene-Specific Analysis** (query_variants_by_gene): For targeted gene panels, cancer genes (BRCA1/2, TP53), pharmacogenes (CYP2D6, CYP2C19)
2. **Chromosomal Analysis** (query_variants_by_chromosome): For chromosomal abnormalities, CNV analysis, specific genomic regions
3. **Population Frequency Analysis** (analyze_allele_frequencies): For rare disease variants, population genetics, allele frequency comparisons with 1000 Genomes
4. **Sample Comparison** (compare_sample_variants): For family studies, cohort analysis, population stratification
5. **Dynamic Queries** (execute_dynamic_genomics_query): For complex clinical questions requiring custom SQL

CLINICAL DECISION SUPPORT:
- Always apply quality filtering (qual > 30 AND PASS filters)
- Prioritize variants by clinical significance and functional impact
- Include 1000 Genomes frequency data for population context
- Provide actionable clinical interpretations

TOOL SELECTION STRATEGY:
1. **For specific genes**: Use query_variants_by_gene (e.g., "BRCA1 variants", "CYP2D6 pharmacogenomics")
2. **For chromosomal regions**: Use query_variants_by_chromosome (e.g., "chromosome 17 variants", "chr13:32000000-33000000")
3. **For frequency analysis**: Use analyze_allele_frequencies (e.g., "rare variants", "population frequencies")
4. **For cohort studies**: Use compare_sample_variants (e.g., "compare samples", "family analysis")
5. **For complex queries**: Use execute_dynamic_genomics_query

EXECUTION FLOW:
1. Understand the user query
2. Select the MOST APPROPRIATE single tool
3. Call the tool with proper parameters
4. Present the results immediately
5. DO NOT call multiple tools unless user explicitly requests for detailed analysis

WHAT NOT TO DO:
- Do not call multiple analysis tools in sequence
- Do not apologize for normal "no results found" scenarios
- Do not add unnecessary conversational elements
- Do not continue analysis after receiving complete results with summary statistics

Remember: Focus on clinically actionable insights that can inform patient care, genetic counseling, and treatment decisions.
"""

# Create the orchestrator
try:
    orchestrator = Agent(
        model=bedrock_model,
        system_prompt=agent_instruction,
        callback_handler=None,
        # associate sub-agents to supervisor
        tools=[vcf_supervisor_agent]
    )
    print("✅ Orchestrator created successfully")
except Exception as e:
    print(f"❌ Error creating orchestrator: {e}")
    orchestrator = None

app = BedrockAgentCoreApp()

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    Invoke the agent with streaming capabilities
    This function demonstrates how to implement streaming responses
    with AgentCore Runtime using async generators
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    
    if orchestrator is None:
        error_response = {"error": "Orchestrator not initialized", "type": "initialization_error"}
        print(f"Initialization error: {error_response}")
        yield error_response
        return
    
    try:
        # Stream each chunk as it becomes available
        async for event in orchestrator.stream_async(user_input):
            if "data" in event:
                yield event["data"]
    except Exception as e:
        # Handle errors gracefully in streaming context
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()

Overwriting vcf_agent_marker.py


In [13]:
# Deploy the agent to Bedrock AgentCore
deployment_name="genomics_store_agent_6"

iam = boto3.client('iam')
agentcore_iam_role = iam.get_role(RoleName='genomics-vep-pipeline-agent-role')['Role']['Arn']
agentcore_iam_role

'<YOUR_AGENT_ROLE_ARN>'

In [ ]:
#pip install git+https://github.com/aws/bedrock-agentcore-starter-toolkit.git

### Configure AgentCore Runtime deployment¶

During the configure step, your docker file will be generated based on your application code.

In [14]:
import boto3
import os
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from botocore.config import Config
boto_session = Session()
region = boto_session.region_name

client = boto3.client(service_name='bedrock-runtime', 
                      region_name='<YOUR_REGION>'
                      )

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="vcf_agent_marker.py",
    execution_role=agentcore_iam_role,
    auto_create_ecr=True,
    requirements_file="runtime_requirements.txt",
    region=region,
    agent_name="vcf_agent_supervisor",
    disable_otel=True
)

Entrypoint parsed: file=/home/ec2-user/SageMaker/Final_deployment/vcf_agent_marker.py, bedrock_agentcore_name=vcf_agent_marker
Configuring BedrockAgentCore agent: vcf_agent_supervisor


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/ec2-user/SageMaker/Final_deployment/Dockerfile
Generated .dockerignore: /home/ec2-user/SageMaker/Final_deployment/.dockerignore
Keeping 'vcf_agent_supervisor' as default agent
Bedrock AgentCore configured: /home/ec2-user/SageMaker/Final_deployment/.bedrock_agentcore.yaml


## Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [15]:
launch_result = agentcore_runtime.launch(
     auto_update_on_conflict=True
)
launch_result

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'vcf_agent_supervisor' to account <YOUR_ACCOUNT_ID> (<YOUR_REGION>)
Starting CodeBuild ARM64 deployment for agent 'vcf_agent_supervisor' to account <YOUR_ACCOUNT_ID> (<YOUR_REGION>)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: vcf_agent_supervisor
✅ ECR repository available: <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_REGION>.amazonaws.com/bedrock-agentcore-vcf_agent_supervisor
Using execution role from config: <YOUR_AGENT_ROLE_ARN>
✅ Execution role validation passed: <YOUR_AGENT_ROLE_ARN>
Preparing C

✅ Reusing existing ECR repository: <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_REGION>.amazonaws.com/bedrock-agentcore-vcf_agent_supervisor


Getting or creating CodeBuild execution role for agent: vcf_agent_supervisor
Role name: AmazonBedrockAgentCoreSDKCodeBuild-<YOUR_REGION>-b5a64a27c2
Reusing existing CodeBuild execution role: <YOUR_CODEBUILD_ROLE_ARN>
Using .dockerignore with 44 patterns
Uploaded source to S3: vcf_agent_supervisor/20250910-142213.zip
Updated CodeBuild project: bedrock-agentcore-vcf_agent_supervisor-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.1s
🔄 PROVISIONING started (total: 5s)
✅ PROVISIONING completed in 5.0s
🔄 DOWNLOAD_SOURCE started (total: 10s)
✅ DOWNLOAD_SOURCE completed in 5.0s
🔄 PRE_BUILD started (total: 15s)
✅ PRE_BUILD completed in 5.0s
🔄 BUILD started (total: 20s)
✅ BUILD completed in 90.7s
🔄 POST_BUILD started (total: 111s)
✅ POST_BUILD completed in 25.2s
🔄 COMPLETED started (total: 136s)
✅ COMPLETED completed in 0.0s
🎉 CodeBuild completed successfully in 2m 16s
CodeBuild completed s

LaunchResult(mode='codebuild', tag='bedrock_agentcore-vcf_agent_supervisor:latest', env_vars=None, port=None, runtime=None, ecr_uri='<YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_REGION>.amazonaws.com/bedrock-agentcore-vcf_agent_supervisor', agent_id='vcf_agent_supervisor-9Y2ejKDXJE', agent_arn='<YOUR_BEDROCK_AGENTCORE_ARN>', codebuild_id='<YOUR_CODEBUILD_PROJECT_ID>', build_output=None)

#### Interactive Genomics Chat Interface

In [16]:
invoke_response = agentcore_runtime.invoke({"prompt": "Howmany patients are there in present cohort?"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll help you determine the number of patients in the present cohort. Let me query the VCF database for this 
information.

Based on the query results, there are **2 unique samples/patients** in the current cohort. 

This is the total number of individual patients whose genomic data is available for analysis in the system. With 
this cohort size, you can perform individual variant analysis on each patient or comparative analyses between the 
two samples, though population-level statistical analyses would be limited due to the small sample size.

Would you like to explore any specific genomic information about these patients, such as variants in particular 
genes, chromosomal regions, or comparisons between the two samples?

In [17]:
invoke_response = agentcore_runtime.invoke({"prompt": "What are the key genomics aberrations linked with heart disease conditions in patient NA21135?"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll help you identify key genomic aberrations linked with heart disease conditions in patient NA21135. Let me 
query the VCF database for this information.

# Cardiac Genomic Analysis for Patient NA21135

## Summary of Findings
Based on the comprehensive analysis of cardiac-related genes in patient NA21135, no clinically significant genomic 
aberrations linked to heart disease were identified.

## Detailed Results:
1. **No Pathogenic or Likely Pathogenic Variants** were found in any major cardiac genes.
2. **No Variants of Uncertain Significance (VUS)** were detected in heart disease-associated genes.
3. **No High or Moderate Impact Variants** that would suggest functional consequences in cardiac pathways.

## Benign Variants Detected:
Several benign variants were identified in important cardiac genes, including:

- **KCNQ1** (Long QT syndrome 1):
  - Multiple benign intronic variants (chr11:2776089:A>G, chr11:2670240:A>G, chr11:2664716:C>T, chr11:2583206:A>G)

- **JUP** (Arrhythmogenic right ventricular dysplasia 12):
  - chr17:41757392:T>C (intronic variant)

- **PRKAG2** (Hypertrophic cardiomyopathy 6):
  - chr7:151556678:C>A (3' UTR variant)

- **RYR2**, **PKP2**, **MYBPC3**, and **LMNA**:
  - Various benign intronic variants

## Clinical Interpretation:
The genomic profile of patient NA21135 does not show evidence of genetic predisposition to common inherited cardiac
conditions such as:
- Hypertrophic cardiomyopathy
- Dilated cardiomyopathy
- Arrhythmogenic right ventricular cardiomyopathy
- Long QT syndrome
- Brugada syndrome

## Clinical Considerations:
- These genomic findings should be correlated with the patient's clinical presentation and family history
- If cardiac symptoms are present despite negative genetic findings, consider:
  - Evaluation of other genetic factors not covered in this analysis
  - Expanded gene panel testing if clinically indicated
  - Regular cardiac monitoring based on clinical presentation

Would you like me to provide more detailed information about any specific cardiac gene or condition?

In [18]:
 invoke_response = agentcore_runtime.invoke({"prompt": "Howmany benign variants are in NA21144?"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll help you determine how many benign variants are present in patient NA21144. Let me query the VCF database for 
this information.

Based on the analysis of the genomic data for patient NA21144, there are **41,999 benign variants** identified.

These benign variants:
- Are classified specifically with "Benign" clinical significance in their annotations
- Pass quality filters (PASS filter status)
- Have quality scores greater than 30, ensuring reliable variant calls

This represents the normal genetic variation in this patient that is not associated with disease or adverse health 
outcomes. These benign variants are typically common in the general population and do not disrupt gene function.

Would you like me to provide more specific information about these benign variants, such as their distribution 
across chromosomes or within specific genes of interest?

In [20]:
invoke_response = agentcore_runtime.invoke({"prompt": "Analyze patient NA21135 and provide individual risk stratification"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll help you analyze patient NA21135 and provide risk stratification. To do this effectively, I need to query the 
genomic variant data for this specific patient.

# Risk Stratification Report: Patient NA21135

## Overall Risk Assessment
Based on comprehensive genomic analysis, patient NA21135 presents with **LOW GENETIC RISK** for hereditary 
conditions across all analyzed disease categories.

## Key Findings
- **No pathogenic or likely pathogenic variants** identified in any analyzed genes
- **No variants of uncertain significance (VUS)** detected
- Multiple benign variants found in clinically relevant genes

## Disease-Specific Risk Assessment

### Hereditary Cancer Risk: LOW
- **Breast/Ovarian Cancer**: No pathogenic BRCA1/BRCA2 variants detected
- **Lynch Syndrome**: No pathogenic MSH2 variants detected
- **Li-Fraumeni Syndrome**: No pathogenic TP53 variants detected

### Cardiovascular Risk: LOW
- **Familial Hypercholesterolemia**: No pathogenic LDLR variants detected
- **Pharmacogenomic Finding**: APOE variant (chr19:44905578:T>G) may affect warfarin response

## Clinical Implications

1. **Cancer Screening**: Follow standard age-appropriate cancer screening guidelines; no indication for enhanced 
surveillance based on genomic findings

2. **Cardiovascular Management**: 
   - Standard lipid monitoring recommended
   - Consider APOE variant if warfarin therapy is needed (may require dosage adjustment)

3. **Family Testing**: Not indicated based on current findings

## Recommendations

1. **Clinical Correlation**: Integrate these genomic findings with family history and other clinical risk factors

2. **Follow-up**: Standard preventive care; no specialized genetic follow-up required unless family history 
suggests otherwise

3. **Medication Considerations**: Document the APOE warfarin response variant in the patient's medical record for 
future reference

This risk stratification is based on current genomic knowledge and available variant classifications. Risk 
assessment should be periodically reviewed as variant interpretations may evolve over time.

In [22]:
invoke_response = agentcore_runtime.invoke({"prompt": "which are the major drug related impactful variant pathway enriched in the patients in the cohort and tell me the allele frequecy reported here and 1000 genome(1000g)"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll analyze the drug-related variants in this patient and compare their allele frequencies with the 1000 Genomes 
Project data.

# Pharmacogenomic Pathway Analysis: Patient NA21135

## Major Drug-Related Impactful Variant Pathways

Based on the analysis of patient NA21135, the following pharmacogenomic pathways show significant variants:

### 1. CYP2D6 Metabolic Pathway (Most Significant)
- **Key Variant**: chr22:42128944:C>T (splice_acceptor_variant)
- **Allele Frequency**:
  - **Patient**: Present (heterozygous)
  - **1000G**: 43.8%
- **Affected Drugs**:
  - Antidepressants (fluoxetine, paroxetine)
  - Opioid analgesics (codeine, tramadol)
  - Antipsychotics (risperidone, aripiprazole)
  - Breast cancer therapy (tamoxifen)
  - Movement disorder treatment (deutetrabenazine)

### 2. Drug Transport Pathway (ABCB1/P-glycoprotein)
- **Key Variant**: chr7:87600184:T>C (5_prime_UTR_variant)
- **Allele Frequency**:
  - **Patient**: Present
  - **1000G**: 100% (common variant)
- **Affected Drugs**:
  - Anticancer agents
  - Immunosuppressants
  - Analgesics (including tramadol)
  - Cardiovascular drugs

### 3. Catecholamine Metabolism Pathway (COMT)
- **Key Variant**: chr22:19962904:A>G (intron_variant)
- **Allele Frequency**:
  - **Patient**: Present
  - **1000G**: 100% (common variant)
- **Affected Drugs**:
  - Pain medications
  - Psychiatric medications

## Allele Frequency Comparison Summary

| Gene   | Variant Position    | Variant Type        | Patient AF | 1000G AF | Clinical Significance |
|--------|---------------------|---------------------|------------|----------|----------------------|
| CYP2D6 | chr22:42128944:C>T  | Splice acceptor     | Present    | 43.8%    | Likely benign/drug response |
| ABCB1  | chr7:87600184:T>C   | 5' UTR variant      | Present    | 100%     | Drug response        |
| COMT   | chr22:19962904:A>G  | Intronic variant    | Present    | 100%     | Benign               |

## Clinical Interpretation

The most clinically relevant pathway enrichment is in the **CYP2D6 metabolic pathway**, which affects approximately
25% of commonly prescribed medications. The splice acceptor variant (chr22:42128944:C>T) is present in the patient 
but is not rare in the general population (43.8% in 1000G).

While the patient carries variants in other important pharmacogenes (CYP2C9, CYP2C19, CYP3A4/5, DPYD, SLCO1B1), 
these are common intronic variants without significant predicted impact on drug metabolism.

The CYP2D6 pathway enrichment suggests potential for altered metabolism of CYP2D6 substrates, which may require 
clinical monitoring when prescribing affected medications, particularly those with narrow therapeutic indices.

In [23]:
invoke_response = agentcore_runtime.invoke({"prompt": "Can you check howmany variants are present for BRCA family of genes in patient 'NA21144' and any clinical significance need to be known?"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll check for BRCA family gene variants in patient NA21144 and their clinical significance.

# BRCA Gene Variant Analysis for Patient NA21144

## Summary of BRCA Variants
Patient NA21144 has a total of **255 BRCA family variants**:
- **202 variants** in BRCA1
- **53 variants** in BRCA2

## Clinical Significance Breakdown
- **Pathogenic variants**: None detected
- **Likely pathogenic variants**: None detected
- **Variants of uncertain significance (VUS)**: 1 variant
- **Benign/Likely benign variants**: 219 variants
- **Unclassified variants**: 35 variants

## Key Variants by Impact Level

### MODERATE Impact Variants (Missense)
Two missense variants in BRCA2, both classified as benign:

1. **BRCA2 chr13:32332591:A>C**
   - Clinical significance: Benign
   - Associated with: Hereditary breast ovarian cancer syndrome
   - Population frequency: Common (1.0 in 1000 Genomes)

2. **BRCA2 chr13:32355249:T>C**
   - Clinical significance: Benign
   - Associated with: Breast ductal adenocarcinoma, Hereditary breast ovarian cancer
   - Population frequency: Common (1.0 in 1000 Genomes)

### LOW Impact Variants (Synonymous)
Two synonymous variants in BRCA2, both classified as benign:

1. **BRCA2 chr13:32340867:G>C**
   - Clinical significance: Benign
   - Associated with: Familial breast cancer
   - Population frequency: Common (1.0 in 1000 Genomes)

2. **BRCA2 chr13:32338917:A>G**
   - Clinical significance: Benign
   - Associated with: Fanconi anemia, Hereditary breast ovarian cancer
   - Population frequency: Common (1.0 in 1000 Genomes)

## Clinical Significance to Note

The most important finding is the presence of **one variant of uncertain significance (VUS)**. While specific 
details of this VUS weren't highlighted in the top results, its presence has clinical implications:

1. **Not immediately actionable**: VUS findings alone don't warrant changes in medical management
2. **Requires monitoring**: The classification may change as more evidence becomes available
3. **Documentation needed**: Should be recorded in the patient's medical record
4. **Genetic counseling recommended**: To explain implications and limitations of this finding

## Recommendations

1. **Standard cancer screening**: Follow age-appropriate guidelines as there are no pathogenic BRCA variants
2. **Genetic counseling referral**: To discuss the VUS and its implications
3. **Periodic reassessment**: The VUS should be reevaluated every 1-2 years as classifications may change
4. **Family history evaluation**: Continue to monitor family cancer history for comprehensive risk assessment

The absence of pathogenic BRCA variants is reassuring, though the presence of a VUS warrants appropriate follow-up 
and monitoring.

In [20]:
invoke_response = agentcore_runtime.invoke({"prompt": "Whats the frequency of chr13:32332591 in BRCA2 variant in this cohort and 1000 genome cohort(1000g)"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll check the frequency of the BRCA2 variant chr13:32332591:A>C in this cohort and compare it with the 1000 
Genomes (1000G) cohort. Let me query the database for this specific information.

# Frequency Analysis of BRCA2 Variant chr13:32332591:A>C

## Cohort Frequency
- **Current Cohort**: Found in 2 out of 2 samples (100%)
  - Present in both NA21135 and NA21144
  - High quality variant calls with quality scores of 164.32 and 139.82
  - Good sequencing depth (39x and 31x coverage)

## 1000 Genomes (1000G) Frequency
- **1000G Allele Frequency**: 1.0 (100%)
  - This indicates the variant is present in all individuals in the 1000 Genomes population

## Variant Details
- **Gene**: BRCA2
- **Genomic Position**: Chromosome 13, position 32332591
- **Nucleotide Change**: A>C
- **Consequence**: Missense variant
- **Impact**: MODERATE
- **Clinical Significance**: Benign (according to ClinVar)

## Clinical Interpretation
This BRCA2 variant (chr13:32332591:A>C) is a common polymorphism that appears to be present in 100% of both your 
cohort and the 1000 Genomes reference population. Its universal presence in the 1000G database and classification 
as "Benign" in ClinVar confirms that this is a common, non-pathogenic variant.

The variant has been evaluated in the context of multiple conditions including:
- Hereditary breast ovarian cancer syndrome
- Breast-ovarian cancer, familial, susceptibility to, 2
- Fanconi anemia complementation group D1

Despite being a missense variant with moderate predicted impact, its high frequency in the general population 
strongly supports its benign classification. This variant is not considered clinically actionable for cancer risk 
assessment.

In [22]:
invoke_response = agentcore_runtime.invoke({"prompt": "Analyze chromosome 17 variants in patient NA21144"})

Invoking BedrockAgentCore agent 'vcf_agent_supervisor' via cloud endpoint


I'll analyze the chromosome 17 variants in patient NA21144. Let me query the database for this information.

# Chromosome 17 Variant Analysis for Patient NA21144

## Overview
I've analyzed all variants on chromosome 17 for patient NA21144. Chromosome 17 is clinically significant as it 
contains several important genes including BRCA1 (breast cancer susceptibility gene) and TP53 (tumor suppressor 
gene).

## Variant Summary

### Total Variants: 132,122

### Impact Distribution:
- **MODIFIER**: 130,835 (99.03%) - primarily non-coding variants
- **LOW**: 788 (0.60%) - synonymous or splice region variants
- **MODERATE**: 477 (0.36%) - missense variants
- **HIGH**: 22 (0.02%) - variants with potentially severe impact (stop gained, frameshift, etc.)

### Top Genes with Variants:
1. ASIC2: 1,896 variants
2. CEP112: 1,665 variants
3. RBFOX3: 1,539 variants
4. UBBP4: 1,163 variants
5. SLC39A11: 1,082 variants
6. BCAS3: 956 variants
7. CA10: 923 variants
8. RPTOR: 819 variants
9. PRKCA: 796 variants
10. ABR: 777 variants

## Clinically Important Genes

### BRCA1 Gene (202 variants)
- **Clinical Classifications**:
  - Benign: 185 variants
  - Benign/Likely benign: 2 variants
  - Uncertain significance: 2 variants
  - No classification: 13 variants
- **Notable variant**: chr17:43044803:CTT>C (3' UTR variant with uncertain significance)
- **No pathogenic or likely pathogenic BRCA1 variants** were identified

### TP53 Gene (35 variants)
- **Key variants**:
  - chr17:7676153:G>C (missense variant, classified as Benign)
  - chr17:7668835:GA>G (3' UTR variant, uncertain significance)
- Most variants are intronic or downstream gene variants
- **No pathogenic or likely pathogenic TP53 variants** were identified

## Variants of Uncertain Significance (VUS)
Two VUS warrant monitoring:
1. chr17:7668835:GA>G in TP53 (3' UTR variant)
2. chr17:43044803:CTT>C in BRCA1 (3' UTR variant)

## Quality Assessment
- The majority of variants have quality scores above 30, indicating good confidence in variant calls
- Sequencing depth ranges from 3x to 167x, with most variants having adequate coverage

## Population Context
- Most variants have high allele frequencies in the 1000 Genomes Project data, suggesting they are common 
polymorphisms
- No rare variants of clinical significance were identified in the key cancer-related genes

## Clinical Interpretation
Patient NA21144 does not appear to have any clinically actionable variants in the major cancer predisposition genes
on chromosome 17 (BRCA1 and TP53). All identified variants in these genes are either benign or of uncertain 
significance, with most being common polymorphisms in the general population.

## Recommendations
1. No immediate clinical action is required based on the chromosome 17 variant profile
2. Consider periodic reassessment of the VUS as knowledge of variant significance evolves
3. Standard cancer screening protocols should be followed based on the patient's personal and family history
4. If there is a strong family history of cancer, consider comprehensive genetic counseling and testing for other 
cancer predisposition genes

## Integrate Agentcore Runtime in Streamlite app

In [21]:
!pip install -r streamlit_requirements.txt

In [2]:
# !pip install --upgrade --force-reinstall awscli

  Using cached botocore-1.40.27-py3-none-any.whl.metadata (5.7 kB)
  Using cached docutils-0.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached s3transfer-0.14.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached rsa-4.7.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 133.9 MB/s eta 0:00:00
Using cached botocore-1.40.27-py3-none-any.whl (14.0 MB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
Using cached docutils-0.19-py3-none-a

In [3]:
!aws --version

aws-cli/1.42.27 Python/3.10.14 Linux/5.10.240-238.959.amzn2.x86_64 awscrt/0.26.1 botocore/1.40.27


In [ ]:
!streamlit run genomics_streamlit_enhanced.py --server.port 8501 --theme.primaryColor "#667eea"




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.16.108.147:8501
  External URL: http://34.236.202.207:8501



## Use your sagemaker link: https://\<sagemaker-notebook-instance-name\>.notebook.\<region_name\>.sagemaker.aws/proxy/8501/ in the web-browser to chat with your agent